In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc
import operator

In [2]:
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
sns.set_context('talk')

In [5]:
import average_precision

In [6]:
import datetime

# Reading

In [7]:
views = pd.read_csv('data/train_category_views.csv', parse_dates=['day'])
clicks = pd.read_csv('data/train_clicks.csv', parse_dates=['day'])[views.columns]

In [8]:
test_users = pd.read_csv('data/test_users.csv')

# Prediction 

In [9]:
number_of_categories = max(max(clicks.category_id), max(views.category_id))
number_of_users = max(max(clicks.user_id), max(views.user_id))

Будем предсказывать только по последней неделе

In [10]:
last_clicks = clicks[(datetime.date(2016, 9, 19) < clicks.day) & (clicks.day < datetime.date(2016, 9, 26))]

In [11]:
user_clicks = np.zeros((number_of_users + 1, number_of_categories + 1))
for row in clicks.iterrows():
    _, row = row
    user_clicks[row['user_id'], row['category_id']] += np.exp((row.day.dayofyear - 269) / 7)

Добавим к каждому пользователю немного от среднего пользователя

In [57]:
average_clicks = user_clicks.sum(axis=0)

In [58]:
average_clicks /= user_clicks.shape[0]

In [59]:
user_clicks += average_clicks / 10

Ещё добавим информацию из views так же по последней неделе

In [20]:
last_views = views[(datetime.date(2016, 9, 19) < views.day) & (views.day < datetime.date(2016, 9, 26))]

In [61]:
for row in views.iterrows():
    _, row = row
    user_clicks[row['user_id'], row['category_id']] += np.exp((row.day.dayofyear - 269) / 7) / 10

Вот, собственно, всё. Предскажем результаты для теста

In [63]:
test_categories = np.argsort(-user_clicks[test_users.user_id.values.reshape(-1), :], axis=1)[:, :5]

In [64]:
test_categories[:5]

array([[1079,  429,  755, 2330,  426],
       [1898,  789,  672,   66,  163],
       [ 200,  672,   66,  163,  424],
       [2138,  160,  672,   66,  163],
       [2273,  672,   66,  163,  424]])

In [65]:
def join_categories(row):
    base_str = ' '.join(map(str, row))
    return base_str + ' '*(25 - len(base_str))

In [66]:
test_users['categories'] = np.apply_along_axis(join_categories, 1, test_categories)

In [67]:
test_users.head()

,user_id,categories
0,8,1079 429 755 2330 426
1,12,1898 789 672 66 163
2,27,200 672 66 163 424
3,39,2138 160 672 66 163
4,40,2273 672 66 163 424


In [68]:
test_users.to_csv('csv/baseline_7_decay.csv', index=None)